In [ ]:
#!pip install /kaggle/input/stable-diffusion-timm/safetensors-0.3.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
#

In [ ]:
#!pip install /kaggle/input/stable-diffusion-timm/timm-0.8.17.dev0-py3-none-any.whl

In [ ]:
#!pip install /kaggle/input/stable-diffusion-timm/ftfy-6.1.1-py3-none-any.whl
#!pip install /kaggle/input/stable-diffusion-timm/open_clip_torch-2.16.0-py3-none-any.whl

In [ ]:
import torchvision

In [ ]:
import numpy as np
import pickle
import pandas as pd
from tqdm import tqdm
import torch
from sklearn.model_selection import train_test_split
from glob import glob
from torch import nn
import torch.optim as optim
from torch.utils.data import DataLoader
from PIL import Image
from pathlib import Path
import open_clip
from transformers import AutoModel, AutoProcessor

In [ ]:
!mkdir -p /kaggle/images/

In [ ]:
images = list(Path('/kaggle/input/stable-diffusion-image-to-prompts/images').glob('*.png')) 
for path in images:
    Image.open(path).save(f'/kaggle/images/{path.stem}.jpg')

In [ ]:
from torchvision import transforms
def _convert_to_rgb(image):
    return image.convert('RGB')
#clip_processor = AutoProcessor.from_pretrained(MODEL)
clip_model, _, _ = open_clip.create_model_and_transforms('convnext_xxlarge', pretrained=None)#"layers": 32
clip_processor = transforms.Compose([
    transforms.Resize(512, interpolation=transforms.InterpolationMode.BICUBIC),
    transforms.CenterCrop(512),
    #transforms.RandomHorizontalFlip(p=1.0),
    _convert_to_rgb, 
    transforms.ToTensor(),
    transforms.Normalize([0.48145466, 0.4578275, 0.40821073], [0.26862954, 0.26130258, 0.27577711]),])

In [ ]:
MODEL = '/kaggle/input/huggingface-vision-models/CLIP-ViT-L-14-laion2B-s32B-b82K/'
#clip_processor = AutoProcessor.from_pretrained(MODEL)
BATCHSIZE=16
SAVE_OPT_CKP = False
SAVE_MODEL_CKP = True
UNFREEZE_START = 18 # set it to lower number when significantly more samples are included.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.backends.cuda.matmul.allow_tf32 = True # allow tf32 on matmul
torch.backends.cudnn.allow_tf32 = True # allow tf32 on cudnn
#clip_model, _, clip_processor = open_clip.create_model_and_transforms('convnext_large_d', pretrained=None)
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        clip = clip_model
        self.vision = clip.visual
        self.fc = nn.Linear(1024, 384)

    def forward(self, x):
        out = self.vision(x)
        return self.fc(out)


def load_pretrained_model():
    model = Net()
    return model.to(device)


class IMGDataset:
    def __init__(self, image_paths, clip_processor=clip_processor):
        self.images = image_paths
        self.input_processor = clip_processor

    def __len__(self):
        return len(self.images)

    def __getitem__(self, item):
        image = Image.open(self.images[item])
        image = self.input_processor(image)
        return image

In [ ]:
images = list(Path('/kaggle/images').glob('*.jpg')) 
model = Net()
model.load_state_dict(torch.load('/kaggle/input/stablediffusion-models/train_convnext_xxlarge_littleboat_on1761k_size512_f0_cv0.6569_ep0_unfree24.pt'))

model.to(device)
model.eval()
test_dataloader = DataLoader(dataset=IMGDataset(images),
                             batch_size=BATCHSIZE, shuffle=False, num_workers=2)
preds = []
with torch.no_grad():
    for batch_images in tqdm(test_dataloader):
        batch_images = batch_images.to(device)

        pred = model(batch_images)
        preds.append(pred.cpu().numpy())


In [ ]:

imgIds = [i.stem for i in images]
EMBEDDING_LENGTH = 384
imgId_eId = [
    '_'.join(map(str, i)) for i in zip(
        np.repeat(imgIds, EMBEDDING_LENGTH),
        np.tile(range(EMBEDDING_LENGTH), len(imgIds)))]

prompt_embeddings = np.vstack(preds).flatten()#predict(images, CFG.model_path, CFG.model_name, CFG.input_size, CFG.batch_size)
submission = pd.DataFrame(
    index=imgId_eId,
    data=prompt_embeddings,
    columns=['val']
).rename_axis('imgId_eId')
submission.to_csv('submission.csv')

In [ ]:
submission